In [118]:
!pip install -q plotly

In [119]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

In [31]:
df = pd.read_csv('apple_stock.csv', index_col=0)
df.head()

,date,x1,x2,x3,x4,x5,x6,x7,x8,x9,...,x12,x13,x14,x15,x16,x17,x18,x19,x20,y
0,2020-09-09,117.320000,113.489998,112.000000,115.360001,115.540001,112.129997,110.339996,106.839996,110.080002,...,108.220001,112.279999,114.959999,114.089996,115.809998,116.790001,113.019997,116.500000,113.160004,115.080002
1,2020-09-10,113.489998,112.000000,115.360001,115.540001,112.129997,110.339996,106.839996,110.080002,111.809998,...,112.279999,114.959999,114.089996,115.809998,116.790001,113.019997,116.500000,113.160004,115.080002,114.970001
2,2020-09-11,112.000000,115.360001,115.540001,112.129997,110.339996,106.839996,110.080002,111.809998,107.120003,...,114.959999,114.089996,115.809998,116.790001,113.019997,116.500000,113.160004,115.080002,114.970001,116.970001
3,2020-09-14,115.360001,115.540001,112.129997,110.339996,106.839996,110.080002,111.809998,107.120003,108.220001,...,114.089996,115.809998,116.790001,113.019997,116.500000,113.160004,115.080002,114.970001,116.970001,124.400002
4,2020-09-15,115.540001,112.129997,110.339996,106.839996,110.080002,111.809998,107.120003,108.220001,112.279999,...,115.809998,116.790001,113.019997,116.500000,113.160004,115.080002,114.970001,116.970001,124.400002,121.099998


In [36]:
df.index = df['date']
df.drop('date', axis=1, inplace=True)

In [45]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(df)

In [50]:
print(len(df)*.8)

400.0


In [79]:
# create train and test sets
X_train = scaled_data[:400, :20]
X_train = np.reshape(X_train, (X_train.shape[0],X_train.shape[1],1))
y_train = scaled_data[:400, -1]

X_test = scaled_data[400:, :20]
y_test = scaled_data[400:, -1]


In [80]:
lstm = Sequential()
lstm.add(LSTM(32, input_shape=(X_train.shape[1], 1), activation='relu',return_sequences=False))
lstm.add(Dense(1))
lstm.compile(loss='mean_squared_error', optimizer='adam')
lstm.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_4 (LSTM)               (None, 32)                4352      
                                                                 
 dense_4 (Dense)             (None, 1)                 33        
                                                                 
Total params: 4,385
Trainable params: 4,385
Non-trainable params: 0
_________________________________________________________________


In [82]:
lstm.fit(X_train, y_train, epochs=10, batch_size=1)

Epoch 1/10
400/400 [==============================] - 2s 2ms/step - loss: 0.0486
Epoch 2/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0052
Epoch 3/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0039
Epoch 4/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0039
Epoch 5/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0035
Epoch 6/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0033
Epoch 7/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0031
Epoch 8/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0029
Epoch 9/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0028
Epoch 10/10
400/400 [==============================] - 1s 2ms/step - loss: 0.0025


In [100]:
X_test_reshaped = np.reshape(X_test, (X_test.shape[0],X_test.shape[1],1))
y_pred = lstm.predict(X_test_reshaped)
test = np.concatenate((X_test, y_pred),axis=1)
test = scaler.inverse_transform(test)

4/4 [==============================] - 0s 3ms/step


In [121]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=train_df.index,y=train_df['y'],
                    mode='lines',
                    name='Training Data'))
fig.add_trace(go.Scatter(x=test_df.index,y=test_df['y'],
                    mode='lines',
                    name='Testing Data'))
fig.add_trace(go.Scatter(x=test_df.index,y=test_df['Predictions'],
                    mode='lines',
                    name='Prediction'))

In [124]:
rms=np.sqrt(np.mean(np.power((test[:,-2]-test[:,-1]),2)))
rms

4.16047341184254